In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA as PCAFromSklearn

# Principal component analysis

Principal component analysis is an unsupervised learning technique. We only work on the feature matrix.


We process the dataset $\hat{\boldsymbol{X}}$ given in form of a matrix with $n$ rows (samples) and $m$ columns (features).

### 1. Centering

We center the data by subtracting the mean of each feature from the corresponding column. We obtain the centered matrix $$\boldsymbol{X} = \hat{\boldsymbol{X}} - \boldsymbol{\mu},$$ where $\boldsymbol{\mu}$ contains the mean of each column of $\hat{\boldsymbol{X}}$.

### 2. Compute covariance matrix

We compute the covariance matrix $$\boldsymbol{\Sigma} = \frac{1}{n-1}\boldsymbol{X}^T \boldsymbol{X}.$$ The covariance matrix $\boldsymbol{\Sigma}$ is a $m \times m$ matrix that is symmetric ($\boldsymbol{\Sigma} = \boldsymbol{\Sigma}^T$) and positive semi-definite ($\boldsymbol{u}^T \boldsymbol{\Sigma} \boldsymbol{u} \leq 0 \quad \forall  \boldsymbol{u} \in \mathbb{R}^m$) by construction. (The symbol $\Sigma$ is not to be confused with the symbol for a sum. It is here simply the uppercase Greek letter sigma often used to denote the covariance matrix.)

### 3. Eigendecomposition of the covariance matrix

We compute the eigenvalues and eigenvectors of $\boldsymbol{\Sigma}$. This is also called the eigendecomposition of $\boldsymbol{\Sigma}$ because $\boldsymbol{\Sigma}$ can be written as $$\boldsymbol{\Sigma} = \boldsymbol{W} \boldsymbol{\Lambda} \boldsymbol{W}^T,$$ where $\boldsymbol{\Lambda}$ is a diagonal matrix containing the eigenvalues of $\boldsymbol{\Sigma}$ on the diagonal and $\boldsymbol{W}$ is a matrix containing the eigenvectors of $\boldsymbol{\Sigma}$ as columns. Eigenvalues are the values that satisfy the following equation: $$\boldsymbol{\Sigma} \boldsymbol{v} = \lambda \boldsymbol{v},$$ where $\lambda$ is an eigenvalue and $\boldsymbol{v}$ is the corresponding eigenvector. The eigenvectors are the directions of the new coordinate system and the eigenvalues are the variances of the data along these directions. The eigenvectors are orthogonal to each other, meaning that the inner product of any two eigenvectors is zero. For a positive semi-definite matrix, the eigenvalues are non-negative.

### 4. Select the first $k$ principal components and build the transformation matrix

We sort the eigenvalues in decreasing order and sort the corresponding eigenvectors accordingly.

We then select the first $k$ eigenvectors to form the reduced matrix $\boldsymbol{W}_k$ (the matrix $\boldsymbol{W}_k$ has $m$ rows and $k$ colums). This matrix is used to transform the data into the new coordinate system. Here is how this works: Take a sample $\boldsymbol{x}^{(i)}$ (a row of $\boldsymbol{X}$) which is a row vector of the form $\boldsymbol{x}^{(i)} = [\boldsymbol{x}_1^{(i)}, \boldsymbol{x}_2^{(i)}, \ldots, \boldsymbol{x}_m^{(i)}]$. The transformed feature vector is given by $$\boldsymbol{z}^{(i)} = \boldsymbol{\phi}_\text{PCA(k)}(\boldsymbol{x}^{(i)}) = \boldsymbol{x}^{(i)}\boldsymbol{W}_k.$$
It has the form $\boldsymbol{z}^{(i)} = [z_1^{(i)}, z_2^{(i)}, \ldots, z_k^{(i)}]$, so it has only $k$ features.
We can do this for all samples and obtain the transformed feature matrix $$\boldsymbol{X}_\text{PCA(k)} := \boldsymbol{Z} = \boldsymbol{X} \boldsymbol{W}_k.$$
This matrix has $n$ rows and $k$ columns. So we have reduced the number of features from $m$ to $k$.

### Remarks

In practice computing the covariance matrix may be expensive and numerically unstable. There are more efficient ways to compute the principal components. The most common one is the singular value decomposition (SVD) of the centered matrix $\boldsymbol{X}$. The SVD of $\boldsymbol{X}$ is given by $$\boldsymbol{X} = \boldsymbol{U} \boldsymbol{S} \boldsymbol{V}^T,$$ where $\boldsymbol{U}$ is a $n \times m$ matrix, $\boldsymbol{S}$ is a $m \times m$ diagonal matrix and $\boldsymbol{V}$ is a $n \times m$ matrix. The columns of $\boldsymbol{V}$ are the eigenvectors of $\boldsymbol{X}^T \boldsymbol{X}$ and the diagonal elements of $\boldsymbol{S}$ are the square roots of the eigenvalues of $\boldsymbol{X}^T \boldsymbol{X}$. The columns of $\boldsymbol{U}$ are the eigenvectors of $\boldsymbol{X} \boldsymbol{X}^T$. You might notice that this is therefore exactly the same as we have done above (apart from the factor $\frac{1}{n-1}$ which can be easily added). However, there are efficient algorithms to compute the SVD of a matrix directly without forming $\boldsymbol{X}^T \boldsymbol{X}$. Moreover, if only the first $k$ principal components are needed, the SVD can be truncated to only keep the first $k$ columns of $\boldsymbol{U}$ and $\boldsymbol{S}$. There are algorithms that can compute the truncated SVD directly without computing the full SVD first saving time and memory.



# Implementation of PCA based on Eigendecomposition

In [ ]:
class PCA:
    """
    Perform principal component analysis (PCA) on a dataset
    """
    def __init__(self, n_components: int):
        """
        Args:
            n_components (int): Number of principal components to keep
        """
        self.n_components = n_components

    def fit(self, X: np.ndarray):
        """
        Args:
            X (np.ndarray): Data matrix of shape (n_samples, n_features)
        """
        # First, we need to center the data
        self.mean_ = X.mean(axis=0)
        X_centered = X - self.mean_

        # Then, we compute the covariance matrix from X as 1/(n-1)*X^T*X
        self.covariance_ = np.dot(X_centered.T, X_centered) / (X_centered.shape[0] - 1)

        # We compute the eigenvalue decomposition of the covariance matrix (Σ = WΛW^T)
        # where each eigenvalue λ satifies the equation Σv = λv
        # where v is a corresponding eigenvector.
        # The function np.linalg.eig returns the eigenvalues as a vector
        # and the eigenvectors as a matrix where each column corresponds to an eigenvector
        self.eigenvalues_, self.eigenvectors_ = np.linalg.eigh(self.covariance_)

        # Sort the eigenvectors by decreasing eigenvalues
        # (or get the index permutation to sort eigenvalues in descending order
        # and apply this permutation to the eigenvalues vector and eigenvectors matrix)
        idx = self.eigenvalues_.argsort()[::-1]
        self.eigenvalues_ = self.eigenvalues_[idx]
        self.eigenvectors_ = self.eigenvectors_[:, idx]

        # Keep only the first n_components eigenvectors (the user asked for in the constructor)
        self.components_ = self.eigenvectors_[:, :self.n_components]

    def transform(self, X: np.ndarray):
        """
        Apply the transformation resulting in a dimensionality reduction of X
        to the first n_components principal components. In other words, project
        the data onto the principal components. In formulas, this is XW where
        X is the data matrix and W is the matrix of principal components (eigenvectors of the covariance matrix).

        Args:
            X (np.ndarray): Data matrix of shape (n_samples, n_features)
        """
        return np.dot(X - self.mean_, self.components_)

# Example on a 2D dataset

In [ ]:
def plot_2d_example_pca(PCA):
    # Create a 2D dataset multi-dimensional Gaussian distribution
    np.random.seed(0)
    X = np.dot([[1.0, 0.4],[0.5, 1]], np.random.randn(2, 500)).T

    # Fit PCA
    pca = PCA(n_components=2)
    pca.fit(X)

    # Plot the data
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))

    # Data with principal components as arrows
    ax[0].scatter(X[:, 0], X[:, 1])
    ax[0].arrow(0, 0, pca.components_[0, 0], pca.components_[0, 1], head_width=0.2, head_length=0.2, fc='k', ec='k')
    ax[0].arrow(0, 0, pca.components_[1, 0], pca.components_[1, 1], head_width=0.2, head_length=0.2, fc='k', ec='k')
    ax[0].text(pca.components_[0, 0] + 0.5, pca.components_[0, 1], 'PC1', ha='center', va='center', fontsize=12, color='black', fontweight='bold')
    ax[0].text(pca.components_[1, 0] + 0.5, pca.components_[1, 1], 'PC2', ha='center', va='center', fontsize=12, color='black', fontweight='bold')
    ax[0].set_xlabel('Feature 1')
    ax[0].set_ylabel('Feature 2')

    # Transform data to principal directions and plot again
    X_pca = pca.transform(X)
    ax[1].scatter(X_pca[:, 0], X_pca[:, 1])
    ax[1].set_xlabel('Principle component 1 (PC1)')
    ax[1].set_ylabel('Principle component 2 (PC2)')

    # Perform another PCA on the transformed data to show that
    # the principal components are orthogonal to the axis now
    pca = PCA(n_components=2)
    pca.fit(X_pca)

    ax[1].arrow(0, 0, pca.components_[0, 0], pca.components_[0, 1], head_width=0.2, head_length=0.2, fc='k', ec='k')
    ax[1].arrow(0, 0, pca.components_[1, 0], pca.components_[1, 1], head_width=0.2, head_length=0.2, fc='k', ec='k')
    ax[1].text(pca.components_[0, 0] + 0.5, pca.components_[0, 1], 'PC1', ha='center', va='center', fontsize=12, color='black', fontweight='bold')
    ax[1].text(pca.components_[1, 0] + 0.5, pca.components_[1, 1], 'PC2', ha='center', va='center', fontsize=12, color='black', fontweight='bold')

    fig.tight_layout()
    plt.show()


In [ ]:
# Plot PCA using the PCA class implemented above
plot_2d_example_pca(PCA)

In [ ]:
# Plot PCA using the PCA class from sklearn
plot_2d_example_pca(PCAFromSklearn)

### Remark

In comparison with the `sklearn` implementation, the implemented PCA class
here might return the principal components with the opposite sign. This is
not an issue as the principal components are only defined up to a sign
and the sign can be flipped without changing the meaning of the principal
components. If $\boldsymbol{\Sigma}$ is the covariance matrix and $\boldsymbol{v}$ is an eigenvector of $\boldsymbol{\Sigma}$,
and $\lambda$ is the corresponding eigenvalue, then $\boldsymbol{\Sigma}\boldsymbol{v} = \lambda \boldsymbol{v}$ and also $\boldsymbol{\Sigma}(-\boldsymbol{v}) = \lambda (-\boldsymbol{v})$.
This means the eigenvalue equation is satisfied for both $\boldsymbol{v}$ and $-\boldsymbol{v}$. (We can
also multiply $\boldsymbol{v}$ by any non-zero scalar and the equation is still satisfied.
Usually eigenvectors are normalized to have a length of $1$ by convention.)

This simply has the effect that the plot is mirrored at the origin.